## Olist Dataset SQL exploratory analysis and visualisation

Thank you for taking the time to read this exploratory analysis SQL/Python Markdown project. My intention for this project is to answer the questions below in SQL, and with relevant python visualisations, followed by a Tableau dashboard for executives to examine and receive actionable ideas from. This project is a means to demonstrate and practice my ability with subqueries, joins, creating views, and SQL/python integration, while also performing some surface analysis of the data provided.

Olist is an online Ecommerce Platform which allows sellers to list their products to the main marketplaces of Brazil e.g. Amazon, Vivo, Casa and Video. How the business works is:
- Sellers list their item through Olist.
- Olist lists the items onto different marketplaces, giving the sellers more visibility.
- Olist charges the business a monthly fee to hold an account, and a commission per sale.

Olist have come up to me, a Data Analyst consultant to answer the questions below, and to provide a dashboard with key KPIs for their executives.

They have provided me this kaggle dataset which is a database of 8 tables, with sales information over 24 months between 2016 and 2018. Dataset available at: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce

1: What is the total revenue generated by Olist, and how has it changed over time?

2: How many orders were placed on Olist, and how does this vary by month or season?

3: What are the most popular product categories on Olist, and how do their sales volumes compare to each other?

4: What is the average order value (AOV) on Olist, and how does this vary by product category or payment method?

6: How many customers have made repeat purchases on Olist, and what percentage of total sales do they account for?

The questions are provided from this medium article, with many thanks: https://medium.com/@tobye070/the-exploratory-data-analysis-on-olist-e-commerce-dataset-cbddd09d936c 

The SQL queries shown in the following are my own, with my interpretation of the questions provided. Please note that the currency is in Brazilian Real, and the conversion rate as of 19th of June is 1 Brazilian Real to 0.3 AUD. In this case study, I have denoted Brazilian real with '$'



Below is the schema supplied kindly by Olist: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce

<div>
<img src = "Olist-schema.png" width = "700"/>
</div>

In [10]:
# importing dependencies into notebook

import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
import plotly.express as px


credentials = service_account.Credentials.from_service_account_file('totemic-studio-372000-56c1543a7106.json')
client = bigquery.Client(credentials=credentials)

# testing dataframes and bigquery connection.
query = """
SELECT * 
FROM totemic-studio-372000.olist_data_set.olist_orders_dataset
LIMIT 5
"""

df = client.query(query).to_dataframe()
display(df)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_time
0,7a4df5d8cff4090e541401a20a22bb80,725e9c75605414b21fd8c8d5a1c2f1d6,created,2017-11-25 11:10:00+00:00,NaT,NaT,NaT,2017-12-12 00:00:00+00:00,0.0
1,35de4050331c6c644cddc86f4f2d0d64,4ee64f4bfc542546f422da0aeb462853,created,2017-12-05 01:07:00+00:00,NaT,NaT,NaT,2018-01-08 00:00:00+00:00,0.0
2,b5359909123fa03c50bdb0cfed07f098,438449d4af8980d107bf04571413a8e7,created,2017-12-05 01:07:00+00:00,NaT,NaT,NaT,2018-01-11 00:00:00+00:00,0.0
3,dba5062fbda3af4fb6c33b1e040ca38f,964a6df3d9bdf60fe3e7b8bb69ed893a,created,2018-02-09 17:21:00+00:00,NaT,NaT,NaT,2018-03-07 00:00:00+00:00,0.0
4,90ab3e7d52544ec7bc3363c82689965f,7d61b9f4f216052ba664f22e9c504ef1,created,2017-11-06 13:12:00+00:00,NaT,NaT,NaT,2017-12-01 00:00:00+00:00,0.0


**1: What is the total revenue generated by Olist, and how has it changed over time?**

In [11]:
# Finding the monthly revenue of Olist
query = '''
WITH cte AS (
  SELECT CONCAT(EXTRACT(MONTH FROM o.order_purchase_timestamp), '-', EXTRACT(YEAR FROM o.order_purchase_timestamp)) AS monthyear, p.payment_value AS payment
  FROM `totemic-studio-372000.olist_data_set.olist_orders_dataset` o
  JOIN `totemic-studio-372000.olist_data_set.olist_order_payments_dataset` p
    ON o.order_id = p.order_id
)

SELECT PARSE_DATE('%m-%Y', monthyear) AS month_start, round(SUM(payment),2) AS total_payment
FROM cte
GROUP BY month_start
order by month_start
'''
df = client.query(query).to_dataframe()
display(df)


,month_start,total_payment
0,2016-09-01,252.24
1,2016-10-01,59090.48
2,2016-12-01,19.62
3,2017-01-01,138488.04
4,2017-02-01,291908.01
5,2017-03-01,449863.60
6,2017-04-01,417788.03
7,2017-05-01,592918.82
8,2017-06-01,511276.38
9,2017-07-01,592382.92


In [16]:
# finding the total revenue during this time period

total_revenue_query = """ 
SELECT sum(payment_value) as total_revenue
FROM totemic-studio-372000.olist_data_set.olist_order_payments_dataset
"""
total_payments = client.query(total_revenue_query).to_dataframe()
display(total_payments)

,total_revenue
0,1.600887e+07


In [19]:
# making the final result prettier (not scientific notation)
total_revenue = total_payments['total_revenue'].values[0]
display(f'${total_revenue:.2f}')


'$16008872.12'

In [20]:
fig = px.bar(df, x= 'month_start', y='total_payment', text_auto = True)
fig.show()

Olist was founded in 2015 by Tiago Dalvi and is based in Brazil. With a simple SQL query, we can see that sales have significantly increased from late 2016 to the end of July. Revenue has remained consistent into 2018, with the 2nd quarter of 2018 performing the best.

I note that there is a clear outlier in the data, with no revenue between Oct 2016 to January 2017, possibly due to a data collection issue. What would be interesting to observe is any seasonality or trends which can affect revenue of Olist, but as there is only 24 months of data and with Olist being a newer company, it it difficult to convey any seasonality from this partciular dataset.

**2: How many orders were placed on Olist, and how does this vary by month or season?**

In [24]:
total_orders = '''
WITH cte AS (
  SELECT CONCAT(EXTRACT(MONTH FROM o.order_purchase_timestamp), '-', EXTRACT(YEAR FROM o.order_purchase_timestamp)) AS monthyear, p.payment_value AS payment
  FROM `totemic-studio-372000.olist_data_set.olist_orders_dataset` o
  JOIN `totemic-studio-372000.olist_data_set.olist_order_payments_dataset` p
    ON o.order_id = p.order_id
)

SELECT PARSE_DATE('%m-%Y', monthyear) AS month_start, count(*) as total_orders
FROM cte
GROUP BY month_start
order by month_start
'''

df_order = client.query(total_orders).to_dataframe()
fig_order = px.bar(df_order, x= 'month_start',y='total_orders', text_auto=True)
fig_order.show()

In [28]:
revperorder = '''
WITH cte AS (
  SELECT CONCAT(EXTRACT(MONTH FROM o.order_purchase_timestamp), '-', EXTRACT(YEAR FROM o.order_purchase_timestamp)) AS monthyear, p.payment_value AS payment
  FROM `totemic-studio-372000.olist_data_set.olist_orders_dataset` o
  JOIN `totemic-studio-372000.olist_data_set.olist_order_payments_dataset` p
    ON o.order_id = p.order_id
)
-- revenue per order
SELECT PARSE_DATE('%m-%Y', monthyear) AS month_start, round(sum(payment)/count(*),2) as revenue_per_order
FROM cte
GROUP BY month_start
order by month_start
'''
df_revperorder = client.query(revperorder).to_dataframe()
fig_rpo = px.bar(df_revperorder, x= 'month_start',y= 'revenue_per_order', text_auto=True)
fig_rpo.show()

Not surprisingly, there is a clear correleation with order numbers and revenue. The amount of revenue per order is also steadily around the 145 to 165$ per order, with no clear seasonality or trends. There are some outlier values, like the revenue per order in September 2018, but can be excluded as there were only 16 orders, and for December 2016 there was only one order.

**3: What are the most popular product categories on Olist, and how do their sales volumes compare to each other?**

In [30]:
category_query = '''
with cte as(
SELECT oi.product_id as product_id, p.string_field_1 as category_name
FROM `totemic-studio-372000.olist_data_set.olist_order_items_dataset` oi, `totemic-studio-372000.olist_data_set.product_category_name_translated` p, `totemic-studio-372000.olist_data_set.olist_products_dataset` op
WHERE oi.product_id = op.product_id
AND op.product_category_name = p.string_field_0
)

SELECT category_name, count(*) as count
FROM cte
GROUP BY category_name
ORDER BY count DESC
Limit 5
'''
df_category_sales = client.query(category_query).to_dataframe()
display(df_category_sales)

,category_name,count
0,bed_bath_table,11115
1,health_beauty,9670
2,sports_leisure,8641
3,furniture_decor,8334
4,computers_accessories,7827


In [43]:
category_query_all = '''
with cte as(
SELECT oi.product_id as product_id, p.string_field_1 as category_name
FROM `totemic-studio-372000.olist_data_set.olist_order_items_dataset` oi, `totemic-studio-372000.olist_data_set.product_category_name_translated` p, `totemic-studio-372000.olist_data_set.olist_products_dataset` op
WHERE oi.product_id = op.product_id
AND op.product_category_name = p.string_field_0
)

SELECT category_name, count(*) as count
FROM cte
GROUP BY category_name
ORDER BY count DESC
'''
df_category_sales_all = client.query(category_query_all).to_dataframe()
df_category_sales_all['% of all sales'] = df_category_sales_all['count']*100/df_category_sales_all['count'].sum()
df_category_sales_all['% of all sales'] = df_category_sales_all['% of all sales'].round(decimals = 2)
display(df_category_sales_all)

,category_name,count,% of all sales
0,bed_bath_table,11115,10.01
1,health_beauty,9670,8.71
2,sports_leisure,8641,7.78
3,furniture_decor,8334,7.51
4,computers_accessories,7827,7.05
...,...,...,...
66,arts_and_craftmanship,24,0.02
67,la_cuisine,14,0.01
68,cds_dvds_musicals,14,0.01
69,fashion_childrens_clothes,8,0.01


Over the past 2 years, there has been a total of 71 different categories sold, with Bed_bath_table items being sold the most frequently of all categories. 

**4: What is the average order value (AOV) on Olist, and how does this vary by product category or payment method?**

Please see the average value of orders solution from part 2.

Below, I will further expand this by product category and payment method

In [37]:
# Average order by category
aov_by_cat = '''
with cte as (
  SELECT o.order_id, oi.product_id, cat.string_field_1 as category,o.payment_type, o.payment_value
  FROM `totemic-studio-372000.olist_data_set.olist_order_payments_dataset` o
  LEFT JOIN `totemic-studio-372000.olist_data_set.olist_order_items_dataset` oi ON oi.order_id = o.order_id
  LEFT JOIN `totemic-studio-372000.olist_data_set.olist_products_dataset` pr ON pr.product_id = oi.product_id
  JOIN `totemic-studio-372000.olist_data_set.product_category_name_translated` cat ON pr.product_category_name = cat.string_field_0
)

SELECT category, round(sum(payment_value)/count(category),2) as AOV
FROM cte
GROUP BY category
ORDER BY AOV desc
'''
df_aov_cat = client.query(aov_by_cat).to_dataframe()
display(df_aov_cat)



,category,AOV
0,computers,1268.73
1,fixed_telephony,763.88
2,small_appliances_home_oven_and_coffee,656.79
3,agro_industry_and_commerce,471.15
4,home_appliances_2,464.79
...,...,...
66,fashion_underwear_beach,88.30
67,food,88.27
68,cds_dvds_musicals,85.67
69,flowers,67.06


In [57]:
# total revenue by category
tr_by_cat = '''
with cte as (
  SELECT o.order_id, oi.product_id, cat.string_field_1 as category,o.payment_type, o.payment_value
  FROM `totemic-studio-372000.olist_data_set.olist_order_payments_dataset` o
  LEFT JOIN `totemic-studio-372000.olist_data_set.olist_order_items_dataset` oi ON oi.order_id = o.order_id
  LEFT JOIN `totemic-studio-372000.olist_data_set.olist_products_dataset` pr ON pr.product_id = oi.product_id
  JOIN `totemic-studio-372000.olist_data_set.product_category_name_translated` cat ON pr.product_category_name = cat.string_field_0
)

SELECT category, round(sum(payment_value)/count(category),2) as AOV, sum(payment_value) as total_revenue
FROM cte
GROUP BY category
ORDER BY total_revenue desc
'''
df_tr_cat = client.query(tr_by_cat).to_dataframe()
display(df_tr_cat)

,category,AOV,total_revenue
0,bed_bath_table,144.85,1712553.67
1,health_beauty,166.20,1657373.12
2,computers_accessories,196.16,1585330.45
3,furniture_decor,163.56,1430176.39
4,watches_gifts,230.48,1429216.68
...,...,...,...
66,flowers,67.06,2213.01
67,home_comfort_2,55.18,1710.54
68,cds_dvds_musicals,85.67,1199.43
69,fashion_childrens_clothes,98.21,785.67


In [45]:
# AOV by payment types
aov_by_pt = '''
with cte as (
  SELECT o.order_id, oi.product_id, cat.string_field_1 as category,o.payment_type, o.payment_value
  FROM `totemic-studio-372000.olist_data_set.olist_order_payments_dataset` o
  LEFT JOIN `totemic-studio-372000.olist_data_set.olist_order_items_dataset` oi ON oi.order_id = o.order_id
  LEFT JOIN `totemic-studio-372000.olist_data_set.olist_products_dataset` pr ON pr.product_id = oi.product_id
  JOIN `totemic-studio-372000.olist_data_set.product_category_name_translated` cat ON pr.product_category_name = cat.string_field_0
)

SELECT payment_type, round(sum(payment_value)/count(payment_type),2) as AOV
FROM cte
GROUP BY payment_type
ORDER BY AOV desc
'''
df_aov_pt = client.query(aov_by_pt).to_dataframe()
display(df_aov_pt)

,payment_type,AOV
0,credit_card,180.07
1,boleto,177.48
2,debit_card,150.25
3,voucher,64.74


In [46]:
aov_pie = px.bar(df_aov_pt, y= 'AOV', x = 'payment_type')
aov_pie.show()

In [52]:
order_spread = '''
with cte as (
  SELECT o.order_id, oi.product_id, cat.string_field_1 as category,o.payment_type, o.payment_value
  FROM `totemic-studio-372000.olist_data_set.olist_order_payments_dataset` o
  LEFT JOIN `totemic-studio-372000.olist_data_set.olist_order_items_dataset` oi ON oi.order_id = o.order_id
  LEFT JOIN `totemic-studio-372000.olist_data_set.olist_products_dataset` pr ON pr.product_id = oi.product_id
  JOIN `totemic-studio-372000.olist_data_set.product_category_name_translated` cat ON pr.product_category_name = cat.string_field_0
)

SELECT order_id, AVG(payment_value) as AOV
FROM cte
GROUP BY order_id
ORDER BY AOV desc

'''

df_os = client.query(order_spread).to_dataframe()
display(df_os)


,order_id,AOV
0,03caa2c082116e1d31e67e9ae3700499,13664.080000
1,736e1922ae60d0d6a89247b851902527,7274.880000
2,0812eb902a67711a1cb742b3cdaa65ae,6929.310000
3,fefacc66af859508bf1a7934eab1e97f,6922.210000
4,f5136e38d1a14a4dbd87dff67da82701,6726.660000
...,...,...
97250,bbc7d177c97f61d86a7486d86ed7a8b2,3.776667
97251,d7c320185a2f9b32f5e2370ff3f8bd64,3.166667
97252,ccf804e764ed5650cd8759557269dc13,2.410769
97253,285c2e15bebd4ac83635ccc563dc71f4,1.856818


In [54]:
df_os_box = px.box(df_os, y= 'AOV')
df_os_box.show()

A few observations looking at the spread of the average order values:
- By category, it appears the computers and home appliances and telephony make up the large revenue values for AOV, which makes sense due to the nature of the product, however they do not end up being the top 3 categories in total revenue.
- The top 3 categories in terms of total revenue is bed_bath, computer accessories, and health_beauty. This suggests that there is not a direct correlation between total revenue and AOV. 
- There is a huge spread in average order value, ranging from $1 to $13000. The median AOV is $103.25, which is equivalent to $AUD 30, with Q1-Q3 being $60.26 to $175.33
- Credit card and Boleto are associated with higher AOVs than debit and voucher payments. This is intuitive when considering that Brazil is a largely cash-based society, where 1/6 of their population do not have banks, and many prefer to use Boleto which allows for cash payments online.

**6: How many customers have made repeat purchases on Olist, and what percentage of total sales do they account for?**

In [64]:
# number of repeat customers 
repeat_customers = '''
with cte AS(SELECT o.customer_id, oi.order_id, p.payment_value, rank() over(partition by o.customer_id ORDER BY oi.order_id) as nb_orders
FROM `totemic-studio-372000.olist_data_set.olist_order_items_dataset` oi
LEFT JOIN `totemic-studio-372000.olist_data_set.olist_orders_dataset` o ON oi.order_id = o.order_id
JOIN `totemic-studio-372000.olist_data_set.olist_order_payments_dataset` p ON o.order_id = p.order_id
GROUP BY customer_id, oi.order_id, p.payment_value)

select count(*) as nb_repeat_customers, sum(payment_value) as repeat_customer_revenue
FROM cte
WHERE customer_id IN (SELECT customer_id
FROM cte
GROUP BY customer_id
HAVING count(*)>1)
'''
df_repeat = client.query(repeat_customers).to_dataframe()
df_repeat



,nb_repeat_customers,repeat_customer_revenue
0,6718,465717.79


Out of 99441 distinct customers, only 6718 were repeat customers. This may be for a few reasons, however further clarification must be taken. 